In [1]:
class Model():
  
  def __init__(self,n_samples,n_channels,n_filters,a_content,a_style,learning_rate,num_iters):
    self.n_samples = n_samples
    self.n_channels = n_channels
    self.n_filters = n_filters
    self.a_content = a_content
    self.a_style = a_style
    self.learning_rate = learning_rate
    self.num_iters = num_iters
    self.kernel = None
  
  def get_features(self):

    a_content_tf = np.ascontiguousarray(self.a_content.T[None,None,:,:])
    a_style_tf = np.ascontiguousarray(self.a_style.T[None,None,:,:])

    # filter shape is "[filter_height, filter_width, in_channels, out_channels]"
    std = np.sqrt(2) * np.sqrt(2.0 / ((self.n_channels + self.n_filters) * 11))
    kernel = np.random.randn(1, 11, self.n_channels, self.n_filters)*std
    self.kernel = kernel
    g = tf.Graph()
    with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
        # data shape is "[batch, in_height, in_width, in_channels]",
        x = tf.placeholder('float32', [1,1,n_samples,n_channels], name="x")

        kernel_tf = tf.constant(kernel, name="kernel", dtype='float32')
        conv = tf.nn.conv2d(
            x,
            kernel_tf,
            strides=[1, 1, 1, 1],
            padding="VALID",
            name="conv")

        net = tf.nn.relu(conv)

        content_features = net.eval(feed_dict={x: a_content_tf})
        style_features = net.eval(feed_dict={x: a_style_tf})

    return content_features, style_features
  
  
  def get_style_gram_matrix(self,style_features):
        
    features = np.reshape(style_features, (-1, self.n_filters))
    style_gram = np.matmul(features.T, features) / self.n_samples
    
    return style_gram

  def optimize(self,content_features, style_gram):

    alpha= 0.01
    
    iterations = 10000

    result = None
    with tf.Graph().as_default():

        # Build graph with variable input
        # x = tf.Variable(np.zeros([1,1,N_SAMPLES,N_CHANNELS], dtype=np.float32), name="x")
        x = tf.Variable(np.random.randn(1,1,self.n_samples,self.n_channels).astype(np.float32)*1e-3, name="x")

        kernel_tf = tf.constant(self.kernel, name="kernel", dtype='float32')
        conv = tf.nn.conv2d(
            x,
            kernel_tf,
            strides=[1, 1, 1, 1],
            padding="VALID",
            name="conv")


        net = tf.nn.relu(conv)

        content_loss = alpha * 2 * tf.nn.l2_loss(
                net - content_features)

        style_loss = 0

        _, height, width, number = map(lambda i: i.value, net.get_shape())

        size = height * width * number
        feats = tf.reshape(net, (-1, number))
        gram = tf.matmul(tf.transpose(feats), feats)  / self.n_samples
        style_loss = 2 * tf.nn.l2_loss(gram - style_gram)

         # Overall loss
        loss = content_loss + style_loss

        opt = tf.contrib.opt.ScipyOptimizerInterface(
              loss, method='L-BFGS-B', options={'maxiter': 10000})

        # Optimization
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())

            print('Started optimization.')
            opt.minimize(sess)

            print ('Final loss:', loss.eval())
            result = x.eval()
            
        return result